In [1]:
from image_processing.image_utils import *
import pandas as pd
import os
import shutil

In [2]:
preds_dir = '/home/rmccune/depth_mapping/CB03_all_photos/test_segs'
labels_dir = '/home/rmccune/depth_mapping/CB03_all_photos/test_labls'

In [3]:
create_labels_from_predsegs(preds_dir, labels_dir, use_gpu=False)

Generating labels from predictions:   0%|          | 0/137 [00:00<?, ?it/s]

Generating labels from predictions: 100%|██████████| 137/137 [00:17<00:00,  7.85it/s]


In [5]:
create_labels_from_predsegs(preds_dir, labels_dir, use_gpu=True, batch_size=100)

Generating labels from predictions: 100%|██████████| 137/137 [00:18<00:00,  7.57it/s]


Separate Water Levels into separate csvs per flood event

In [2]:
# Load the abbreviated flood events and full data records
abbr_flood_events = pd.read_csv('/home/rmccune/depth_mapping/CB_03_flood_record/abbr_flood_events.csv')
flood_events = pd.read_csv('/home/rmccune/depth_mapping/CB_03_flood_record/flood_events.csv')

# Ensure the output directory exists
output_dir = 'output_flood_events'
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Convert the start and end time columns to datetime
abbr_flood_events['start_time_UTC'] = pd.to_datetime(abbr_flood_events['start_time_UTC'], utc=True)
abbr_flood_events['end_time_UTC'] = pd.to_datetime(abbr_flood_events['end_time_UTC'], utc=True)

# Format datetime to string for folder naming
abbr_flood_events['start_time_str'] = abbr_flood_events['start_time_UTC'].dt.strftime('%Y%m%d%H%M%S')
abbr_flood_events['end_time_str'] = abbr_flood_events['end_time_UTC'].dt.strftime('%Y%m%d%H%M%S')

# Iterate over each unique flood event in the abbreviated events
for _, row in abbr_flood_events.iterrows():
    flood_event_number = row['flood_event']
    start_time_utc = row['start_time_str']
    end_time_utc = row['end_time_str']
    
    sensor_id = row['sensor_ID']
    
    # Filter the full flood events DataFrame for the current flood event number
    filtered_events = flood_events[flood_events['flood_event'] == flood_event_number]
    
    # Generate a filename based on start_time_UTC and end_time_UTC
    filename = f"{sensor_id}_{start_time_utc}_{end_time_utc}.csv"
    
    # Save the filtered DataFrame to a new CSV file
    filtered_events.to_csv(os.path.join(output_dir, filename), index=False)

print(f"Successfully created CSV files in '{output_dir}' directory.")

Successfully created CSV files in 'output_flood_events' directory.


Move csvs into proper subfolders

In [5]:
# Define the main directory where the output CSV files are located
main_directory = 'output_flood_events'
# Define the parent directory where you want to create subfolders
parent_directory = '/home/rmccune/depth_mapping/CB03_all_photos/flood_events'  # Change this to your actual parent directory

# Iterate over the files in the main directory
for filename in os.listdir(main_directory):
    if filename.endswith('.csv'):
        # Extract the folder name from the filename (without the .csv extension)
        folder_name = filename[:-4]  # Remove '.csv' from the filename
        
        # Create the full path for the subfolder
        subfolder_path = os.path.join(parent_directory, folder_name)
        
        # Create the subfolder if it doesn't exist
        os.makedirs(subfolder_path, exist_ok=True)
        
        # Move the CSV file to the corresponding subfolder
        source_path = os.path.join(main_directory, filename)
        destination_path = os.path.join(subfolder_path, filename)
        shutil.move(source_path, destination_path)

print("CSV files have been moved to their corresponding subfolders.")


CSV files have been moved to their corresponding subfolders.


In [1]:
import numpy as np 
import cupy as cp
import cupyx as cpx
import time
import cv2
import zarr

# from gpu_acc_utils import *
from cupyx.scipy.interpolate import RegularGridInterpolator as reg_interp
from cupyx.scipy.ndimage import binary_closing
from cupyx.scipy.ndimage import label
from skimage.measure import find_contours
import image_processing

In [2]:
file_path = '/home/rmccune/depth_mapping/data/lidar/Job1051007_34077_04_88.laz'

min_x_extent = 712160
max_x_extent = 712230
min_y_extent = 33100
max_y_extent = 33170

grid_gen = image_processing.GridGenerator(file_path, min_x_extent, max_x_extent, min_y_extent, max_y_extent)

In [3]:
resolution = 0.05 # meters

pts_array = grid_gen.create_point_array()
grid_x, grid_y, grid_z = grid_gen.gen_grid(resolution, z=pts_array)

grid_z_gpu = cp.asarray(grid_z)

Directory to store grids already exists: generated_grids


In [4]:
from image_processing.depth_mapper import DepthMapper

In [7]:
zarr_directory_path = '/home/rmccune/depth_mapping/CB03_all_photos/flood_events/CB_03_20220617023112_20220617040940/zarr/labels_rects'
zarr_store_path = '/home/rmccune/depth_mapping/CB03_all_photos/flood_events/CB_03_20220617023112_20220617040940/zarr/depth_maps_95th_ponding_OOP_debug_save'

mapper = DepthMapper(grid_z_gpu)

In [ ]:
mapper.process_depth_maps(zarr_directory_path, zarr_store_path)

In [7]:
import os
import zarr
import matplotlib.pyplot as plt
import cmocean

In [8]:
def plot_depth_maps(zarr_dir, plotting_folder, depth_min=0, depth_max=0.25):

    depth_data = []

    for file_name in os.listdir(zarr_dir):
        if file_name.endswith('_ponding'):
            zarr_store_path = os.path.join(zarr_dir, file_name)
            img_store = zarr.open(zarr_store_path)
            
            depth_map = img_store[:]
            
            print(f"Processing depth map: {file_name}")

            # Overlay the depth map
            im = plt.imshow(depth_map, cmap=cmocean.cm.deep, vmin=depth_min, vmax=depth_max)  # Adjust alpha for transparency
            # print(f"Vmin:{depth_min} Vmax: {depth_max} Max depth: {np.nanmax(depth_map)} Min depth: {np.nanmin(depth_map)}")

            # plt.scatter(max_elev_point_indices[1], max_elev_point_indices[0], c='red', s=10, marker='o')

            # Add a colorbar for the depth map
            cbar = plt.colorbar(im, label='Depth')
            cbar.set_label('Depth (meters)')

            plt.gca().invert_yaxis()
            plt.xlabel('X (cm)')
            plt.ylabel('Y (cm)')

            # Save the figure
            plt.savefig(os.path.join(plotting_folder, file_name), 
                        bbox_inches='tight', pad_inches=0.1, dpi=300)
            
            plt.close()

In [9]:
zarr_store_path = '/home/rmccune/depth_mapping/CB03_all_photos/flood_events/CB_03_20220617023112_20220617040940/zarr/depth_maps_95th_ponding_OOP_debug_save'
plot_depth_maps(zarr_store_path, '/home/rmccune/depth_mapping_updated/plots/depth_maps_95th_ponding_OOP_debug_save')

Processing depth map: CAM_CB_03_20220617023633_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617033032_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617033631_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617035433_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617032434_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617040633_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617034232_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617030630_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617025433_predseg_labels_rectified_depth_map_95_perc_edge_ponding
Processing depth map: CAM_CB_03_20220617040032_predseg_labels_rectified_d

In [5]:
import os
from tqdm import tqdm

In [ ]:
# Path to the main directory containing subfolders
main_directory = '/home/rmccune/depth_mapping/CB03_all_photos/flood_events'

# # Iterate through each subfolder in the main directory
# for subfolder in os.listdir(main_directory):
#     subfolder_path = os.path.join(main_directory, subfolder)
    
#     # Check if it is a directory
#     if os.path.isdir(subfolder_path):
#         labels_rects_zarr_folder = os.path.join(subfolder_path, 'zarr', 'labels_rects')
        
#         # Check if the orig_images and labels folders exist
#         if os.path.exists(labels_rects_zarr_folder):
#             depth_map_zarr_save_dir = os.path.join(subfolder_path, 'zarr', 'depth_maps_95th_ponding')
#             mapper.process_depth_maps(labels_rects_zarr_folder, depth_map_zarr_save_dir)

# Get a list of subfolders
subfolders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]

# Iterate through each subfolder with a progress bar
for subfolder in tqdm(subfolders, desc="Processing flood events", unit="event"):
    subfolder_path = os.path.join(main_directory, subfolder)
    labels_rects_zarr_folder = os.path.join(subfolder_path, 'zarr', 'labels_rects')
    
    # Check if the labels_rects folder exists
    if os.path.exists(labels_rects_zarr_folder):
        # print(f"Processing folder {subfolder}.")
        depth_map_zarr_save_dir = os.path.join(subfolder_path, 'zarr', 'depth_maps_95th_ponding')
        mapper.process_depth_maps(labels_rects_zarr_folder, depth_map_zarr_save_dir)

Processing flood events:   0%|          | 0/55 [00:00<?, ?event/s]

Processing folder CB_03_20230929002604_20230929031404.


Processing flood events:   2%|▏         | 1/55 [00:02<02:40,  2.97s/event]

Processing folder CB_03_20240109225226_20240109232226.


Processing flood events:   4%|▎         | 2/55 [00:04<01:45,  1.99s/event]

...

Processing flood events:  96%|█████████▋| 53/55 [02:23<00:06,  3.22s/event]

Processing folder CB_03_20240109235826_20240110005226.


Processing flood events:  98%|█████████▊| 54/55 [02:24<00:02,  2.77s/event]

Processing folder CB_03_20220615012935_20220615024735.


Processing flood events: 100%|██████████| 55/55 [02:25<00:00,  2.65s/event]
